# Heating Value
This Jupyter Notebook was written by Dan Haworth, borrowing from the tutorial Notebook "heating_value.ipynb" that is available at https://github.com/Cantera/cantera-jupyter/tree/master/thermo. It is intended as a tutorial to be used in parallel with Chapter 2 of the book "An Introduction to Combustion: Concepts and Applications" by Stephen R. Turns and Daniel C. Haworth. That book is referred to as "Turns 4th ed." throughout this Notebook.

This Notebook was last updated 7 February 2020.

The objectives of this tutorial Notebook are to review the concept of heating values, and to show how to compute heating values using Cantera's `Solution()` object, together with properties for liquid water using Cantera's `Water()` object. We will define the gas mixture using gri30.cti.

The reader should be familiar with the material in the previous Notebook (2_4_Enthalpy) before working through this Notebook.

## 1. Definitions

The *heating value* is defined as the enthalpy of a reactant mixture (a specified fuel-oxidizer mixture), minus the enthalpy of the resulting (idealized) product mixture, for the case where the reactants and the products are at the same pressure and temperature:

$$ Heating \ Value \equiv H_{reactants} - H_{products} \ , \ {\rm with \ reactants \ and \ products \ at \ the \ same \ pressure \ and \ temperature} $$

The heating value is conventionally reported as a fuel property. It is a measure of the chemical energy that is available for eventual conversion to sensible energy (heat) in a combustion process, for example.

As in previous tutorial Notebooks, here we are interested primarily in fuels that contain (at most) carbon, hydrogen, and oxygen atoms ($C_xH_yO_z$), burning with our standard approximation for air (21% $O_2$ and 79% $N_2$ on a volume or molar basis). For the products, we consider complete combustion in the sense defined earlier (tutorial Notebook 2_3_Stoichiometry), so that all fuel carbon oxidizes to form $CO_2$ and all fuel hydrogen oxidizes to form $H_2O$. And we again assume that $N_2$ does not participate chemically. For the case of a stoichiometric reactant mixture and complete combustion, the global reaction can be written as:

$$ C_x H_y O_z + (x+y/4-z/2)( O_2 + 3.76 N _ 2 ) \rightarrow x CO_2 + y/2 H_2 O + 3.76 (x+y/4-z/2) N_2 $$

The heating value for a specified fuel can be reported either on a per-unit-mass-of-fuel basis or on a per-unit-mole-of-fuel basis. The numerical value of the heating value is the same for pure oxygen ($O_2$) as the oxidizer as for our usual approximation for air as the oxidizer, since the enthalpy of the $N_2$ is the same in the reactants and in the products (which, for purposes of defining the heating value, are at the same pressure and temperature). 

Moreover, heating values are normally reported at the standard reference temperature (298.15 K) and pressure (1 atm). Since water can be either a liquid or a vapor at the reference temperature and pressure, it is conventional to define two heating values: a *lower heating value* (LHV) corresponding to the product water being in the vapor state, and a *higher heating value* (HHV) corresponding to the product water being in the liquid state. The two values differ by the change in enthalpy associated with condensing the product water from a vapor to a liquid at the reference pressure and temperature: i.e., by the enthalpy of vaporization of the product water. In most cases, the lower heating value of a fuel (rather than the higher heating value) is used as the basis for comparing the relative energy content of different fuels.

With this background, let's see how we can compute heating values using Cantera.

## 2. Computing lower heating value using Cantera

We will start with lower heating values, because only gas-phase properties are needed to calculate those.

As in previous tutorial Notebooks, we again use Cantera's `Solution()` object and GRI-Mech 3.0 to define a gas-phase mixture named "gas1": 

In [1]:
# access modules
import cantera as ct
import numpy as np

# report Cantera version
print("Running Cantera version: {}".format(ct.__version__))

Running Cantera version: 2.4.0


In [2]:
# define an ideal-gas mixture named "gas1" using Cantera's "Solution()" object and GRI-Mech 3.0
gas1 = ct.Solution('gri30.cti')

In the previous tutorial Notebook (2_4_Enthalpy), we saw how to access and work with individual-species and mixture enthalpies. And in the Notebook before that (2_3_Stoichiometry), we saw how to compute (to a first approximation) reactant and product compositions corresponding to stoichiometric (and fuel-lean) reactants.

Let's consider propane fuel ($C_3H_8$), as an example.

In [3]:
fuel = 'C3H8'
air  = 'O2:1.0, N2:3.76'
phi  = 1.0

# set the gas mixture to correspond to a stoichiometric propane-air mixture at the reference temperature and pressure
gas1.TP = 298.15 , ct.one_atm
gas1.set_equivalence_ratio(phi,fuel,air)

# print the full thermochemical state of gas1
gas1()


  gri30:

       temperature          298.15  K
          pressure          101325  Pa
           density         1.20437  kg/m^3
  mean mol. weight         29.4655  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy     -1.4208e+05       -4.187e+06     J
   internal energy     -2.2621e+05       -6.666e+06     J
           entropy          6887.4        2.029e+05     J/K
    Gibbs function     -2.1956e+06       -6.469e+07     J
 heat capacity c_p          1049.6        3.093e+04     J/K
 heat capacity c_v          767.42        2.261e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
                O2       0.201613         0.218947         -26.2751
                N2       0.758065         0.720709           -23.31
              C3H8      0.0403226        0.0603447            -77.6
     [  +50 minor]        

In [4]:
# store the reactant enthalpy on a per-unit-mass-of-mixture basis (J/kg)
h_reac_mass = gas1.enthalpy_mass

h_reac_mass

-142083.21261478376

In [5]:
# store the mass fraction of fuel (C3H8) in the reactants
Y_C3H8_reac = gas1.Y[gas1.species_index('C3H8')]

Y_C3H8_reac

0.06034469442007445

Next, we set the composition of gas1 to correspond to the products of complete combustion, without changing the pressure and temperature. That can be done in various ways. One approach was shown in tutorial Notebook 2_3_Stoichiometry. Following that approach (and for generality, allowing for a fuel molecule that contains C, H, and O atoms): 

In [6]:
x = gas1.n_atoms(fuel,'C') # number of atoms of C in the fuel molecule
y = gas1.n_atoms(fuel,'H') # number of atoms of H in the fuel molecule
z = gas1.n_atoms(fuel,'O') # number of atoms of O in the fuel molecule

x , y , z

(3.0, 8.0, 0.0)

In [7]:
# a is the number of moles of O2 needed per mole of fuel for complete combustion
a = x + y/4. - z/2.
    
# compute the numbers of product moles per mole of fuel, for complete combustion of a stoichiometric reactant mixture
N_CO2 = x
N_H2O = y/2.
N_N2  = 3.76*a

N_CO2 , N_H2O , N_N2

(3.0, 4.0, 18.799999999999997)

In [8]:
# set the gas mixture to correspond to products of complete combustion of stoichiometric propane-air reactants
#   at the reference temperature and pressure
# recall that the specified numbers of moles will be interpreted as relative mole numbers, so that the
#   actual product mole fractions sum to unity
X_prod = {'CO2':N_CO2,'H2O':N_H2O,'N2':N_N2}
gas1.TPX = None , None , X_prod

# print the full thermochemical state of gas1
gas1()


  gri30:

       temperature          298.15  K
          pressure          101325  Pa
           density         1.15769  kg/m^3
  mean mol. weight         28.3234  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy     -2.9392e+06       -8.325e+07     J
   internal energy     -3.0267e+06       -8.573e+07     J
           entropy          7064.4        2.001e+05     J/K
    Gibbs function     -5.0454e+06       -1.429e+08     J
 heat capacity c_p          1084.2        3.071e+04     J/K
 heat capacity c_v          790.67        2.239e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
               H2O       0.155039        0.0986134         -122.126
               CO2       0.116279         0.180678         -186.604
                N2       0.728682         0.720709         -23.3496
     [  +50 minor]        

In [9]:
# store the product enthalpy on a per-unit-mass-of-mixture basis (J/kg)
h_prod_mass = gas1.enthalpy_mass

h_prod_mass

-2939189.2483540904

Note that the product enthalpy is lower than the reactant enthalpy (both on a per-unit-mass basis; mass is conserved in chemical reactions). This tells us that the global reaction considered here is exothermic in the forward direction.

In [10]:
# compute the lower heating value on a per-unit-mass-of-fuel basis
# note that mass is conserved in the conversion of reactants to products, so that we can simply
#   subtract the product enthalpy (per unit mass of product mixture) from the reactant enthalpy
#   (per unit mass of reactant mixture), then divide by the mass fraction of fuel in the reactants
#   to put the heating value on the basis of a unit mass of fuel
lhv_mass = ( h_reac_mass - h_prod_mass ) / Y_C3H8_reac # lower heating value in Joules per kg of fuel

# compute the lower heating value on a per-unit-mole-of-fuel basis
lhv_mole = lhv_mass*gas1.molecular_weights[gas1.species_index('C3H8')] # lower heating value in Joules per kmol of fuel

# convert to kJ/kg and kJ/kmol, from J/kg and J/kmol
lhv_mass = lhv_mass / 1000. # kJ/kg
lhv_mole = lhv_mole / 1000. # kJ/kmol

lhv_mass , lhv_mole

(46352.14516570347, 2043968.2963423461)

The lower heating value per unit mass of fuel is consistent with the value reported in Table B.1 of Appendix B of Turns 4th ed. for vapor $C_3H_8$ fuel, to four significant figures.

## 3. Computing higher heating value using Cantera

To compute higher heating values, liquid water properties are needed, in addition to gas-phase properties.

Properties of liquid water can be accessed using Cantera's [`Water()`](https://cantera.org/documentation/docs-2.4/sphinx/html/cython/importing.html#pure-fluid-phases) object: 

In [11]:
# define a water liquid-gas mixture named "water" using Cantera's "Water" object
water = ct.Water()

In [12]:
# show all available attributes of "water" - here we will only use a couple of them
# you can use "help" to learn about any attributes of interest, including the ones that are used below
dir(water)

['DP',
 'DPX',
 'DPY',
 'HP',
 'HPX',
 'HPY',
 'ID',
 'P',
 'PV',
 'PX',
 'P_sat',
 'SH',
 'SP',
 'SPX',
 'SPY',
 'ST',
 'SV',
 'SVX',
 'SVY',
 'T',
 'TD',
 'TDX',
 'TDY',
 'TH',
 'TP',
 'TPX',
 'TPY',
 'TV',
 'TX',
 'T_sat',
 'UP',
 'UV',
 'UVX',
 'UVY',
 'VH',
 'X',
 'Y',
 '__call__',
 '__class__',
 '__copy__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_full_states',
 '_init_cti_xml',
 '_init_parts',
 '_references',
 'activities',
 'activity_coefficients',
 'add_species',
 'atomic_weight',
 'atomic_weights',
 'basis',
 'chemical_potentials',
 'concentrations',
 'cp',
 'cp_mass',
 'cp_mole',
 'critical_density',
 'critical_pressure',
 'critical_temperature',
 'cv',
 'cv_mass',
 'cv_m

In [13]:
# set the water state to the reference temperature, with a vapor fraction equal to zero (i.e., pure liquid water)
water.TX = 298.15, 0.0

# h_liq is the enthalpy of liquid water (J/kg) at the reference temperature
h_liq = water.h

# set the water state to the reference temperature, with a vapor fraction equal to one (i.e., pure vapor water)
water.TX = 298.15, 1.0

# h_gas is the enthalpy of vapor water (J/kg) at the reference temperature
h_gas = water.h

# h_gas-h_liq is then the enthalpy of vaporization of water (J/kg) at the reference temperature
# the factor of 1000. is included to convert the units to kJ/kg
h_fg = ( h_gas - h_liq ) / 1000.

# the enthalpy of vaporization on a molar basis (kJ/kmol) is found by multiplying by the molecular weight of H2O
hbar_fg = h_fg*gas1.molecular_weights[gas1.species_index('H2O')]

h_fg , hbar_fg

(2442.3120195756946, 43998.934880021625)

The enthalpy of vaporization of water given in Table A.6 of Turns 4th ed. is slightly higher than the value computed here: 44,010 kJ/kmol. The difference is presumably a result of small differences between thermodynamic property values obtained from different sources.

In [14]:
# now we can calculate the higher heating value, which corresponds to the product water being in the liquid state
# N_H2O is the number of moles of H2O formed per mole of C3H8 burned (kmol_h2o/kmol_c3h8), so that
#   m_h2o is the mass of water produced per mass of C3H8 burned (kg_h2o/kg_c3h8)
m_h2o = N_H2O*gas1.molecular_weights[gas1.species_index('H2O')] / gas1.molecular_weights[gas1.species_index('C3H8')]

m_h2o

1.6341679570179235

In [15]:
# hhv_mass is the higher heating value of C3H8 in kJ/kg
# hhv_mole is the higher heating value of C3H8 in kJ/kmol
hhv_mass = lhv_mass + m_h2o*h_fg
hhv_mole = hhv_mass*gas1.molecular_weights[gas1.species_index('C3H8')]

hhv_mass , hhv_mole

(50343.2932091338, 2219964.035862433)

The higher heating value per unit mass of fuel is consistent with the value reported in Table B.1 of Appendix B of Turns 4th ed. for vapor $C_3H_8$ fuel, to four significant figures.

## 4. Heating values for several fuels

Next, we generalize the above example into a function that computes the lower and higher heating values on both a per-unit-mass-of-fuel basis and a per-unit-mole-of-fuel basis, for a fuel molecule that contains (at most) C, H, and O atoms:

In [16]:
# define a function that returns the lower and higher heating values of a fuel on both a per-unit-mass-of-fuel basis
#   and a per-unit-mole-of-fuel basis
# the function takes a single argument: the fuel type (denoted here as "fuel")
# the heating value is computed as the difference between the reactant mixture enthalpy and the product mixture enthalpy,
#   where both the reactants and the products are at the reference temperature and pressure,
#   the reactants are a stoichiometric mixture of the specified fuel and our standard approximation for air,
#   and the products are the products of complete combustion
# with these assumptions, the numerical value of the heating value would be the same for pure O2 oxidizer (rather than air)
# the ideal-gas mixture "gas" must be defined before calling this function
# the water mixture ("water") must be defined before calling this function
# the specified fuel must be one that is available in "gas"
# the specified fuel must contain (at most) C, H, and O atoms

def heating_values(fuel):
    
# set the reference temperature and pressure
    T_ref = 298.15     # K
    P_ref = ct.one_atm # Pa
    
# use our standard approximation for air
# the heating value results would be identical for pure O2 as the oxidizer (and with zero product moles of N2)
    air = 'O2:1.0 N2:3.76'

# set the gas mixture thermochemical state to correspond to stoichiometric fuel-air reactants at the reference T and p
    gas.TP = T_ref , P_ref
    gas.set_equivalence_ratio(1.0, fuel, air)
    
# save the mass fraction of fuel in the reactants and the reactant mass-specific mixture enthalpy (J/kg)
    Y_fuel_reac = gas.Y[gas.species_index(fuel)]
    h_reac_mass = gas.enthalpy_mass

# x, y, and z are the numbers of atoms of carbon, hydrogen, and oxygen in the fuel molecule, respectively
    x = gas.n_atoms(fuel,'C')
    y = gas.n_atoms(fuel,'H')
    z = gas.n_atoms(fuel,'O')
    
# a is the number of moles of O2 needed per mole of fuel for complete combustion
    a = x + y/4. - z/2.
    
# compute the numbers of product moles per mole of fuel, for complete combustion of stoichiometric reactants
    N_CO2 = x
    N_H2O = y/2.
    N_N2  = 3.76*a
    
# reset the mixture thermochemical state to correspond to products of complete combustion
#   of stoichiometric fuel-air reactants at the reference T and p
    X_prod = {'CO2':N_CO2,'H2O':N_H2O,'N2':N_N2}
    gas.TPX = None , None , X_prod
    
# save the product mass-specific mixture enthalpy (J/kg)
    h_prod_mass = gas.enthalpy_mass
    
# compute the lower heating value on a per-unit-mass-of-fuel basis (J/kg)
    lhv_mass = ( h_reac_mass - h_prod_mass ) / Y_fuel_reac 
    
# compute the lower heating value on a per-unit-mole-of-fuel basis (J/kmol)
    lhv_mole = lhv_mass*gas.molecular_weights[gas.species_index(fuel)]
    
# compute the enthalpy of vaporization of water (J/kg) at T_ref
# this doesn't really need to be redone each time that the function is called, but it is included here for completeness
    water.TX = T_ref, 0.0
    h_liq    = water.h
    water.TX = T_ref, 1.0
    h_gas    = water.h
    h_fg     = h_gas - h_liq

# compute the mass of product water per mass of fuel burned
    m_h2o = N_H2O*gas.molecular_weights[gas.species_index('H2O')] / gas.molecular_weights[gas.species_index(fuel)]

# compute the higher heating value on a per-unit-mass-of-fuel basis (J/kg)
    hhv_mass = lhv_mass + m_h2o*h_fg

# compute the higher heating value on a per-unit-mole-of-fuel basis (J/kmol)
    hhv_mole = hhv_mass*gas.molecular_weights[gas.species_index(fuel)]

# return the four heating values - note the units of each, which are given above
    return lhv_mass,lhv_mole,hhv_mass,hhv_mole

# this is the end of the definition of the function "heating_values"

Now we will call the function to compute the heating values for several different fuels.

In [17]:
# define an ideal-gas mixture named "gas" using Cantera's "Solution" object and GRI-Mech 3.0
gas = ct.Solution('gri30.cti')

# define a water liquid-gas mixture named "water" using Cantera's "Water" object
water = ct.Water()

# list the fuels: hydrogen, methane, ethane, propane, acetylene, and methanol
fuels  = ['H2', 'CH4', 'C2H6', 'C3H8', 'C2H2', 'CH3OH']
nfuels = len(fuels)

# define an array to hold four heating values for each fuel
hvs = np.zeros(shape=(nfuels,4))

# print a table header
header1 = 'Fuel            LHV            LHV            HHV             HHV'
header2 = 'Type          (kJ/kg)       (kJ/kmol)       (kJ/kg)        (kJ/kmol)'
print(' {:60s}'.format(header1) )
print(' {:60s}'.format(header2) )

# loop over fuels, and compute heating values
# convert energy units from J to kJ for printing
for i , fueli in enumerate(fuels):
    hvs[i,:] = heating_values(fueli)
    print(' {:8s} {:15f} {:15f}{:15f} {:15f}'.format(fueli,hvs[i,0]/1000.,hvs[i,1]/1000.,hvs[i,2]/1000.,hvs[i,3]/1000.) )

 Fuel            LHV            LHV            HHV             HHV
 Type          (kJ/kg)       (kJ/kmol)       (kJ/kg)        (kJ/kmol)
 H2         119959.822299   241824.606576  141785.989968   285823.541456
 CH4         50026.141167   802557.376464   55511.348809   890555.246224
 C2H6        47510.985335  1428638.225061   51900.688858  1560635.029701
 C3H8        46352.145166  2043968.296342   50343.293209  2219964.035862
 C2H2        48277.308237  1257038.758606   49967.113048  1301037.693486
 CH3OH       21104.011213   676218.103931   23850.326373   764215.973691


For methane, ethane, propane, and acetylene, the lower and higher heating values per unit mass of fuel computed here are consistent with those given in Table B.1 of Turns 4th ed., to within three-to-four significant figures. The values for hydrogen and methanol are consistent with those that can be found in other sources.

On a per-unit-mass-of-fuel basis, the lower heating value of hydrogen is much higher than that of any of the other fuels. The lower heating values of all of the hydrocarbon fuels other than methane are within approximately 10% of one another. With the exception of benzene ($C_6H_6$), that is also the case for the other hydrocarbon fuels listed in Table B.1 of Turns 4th ed. The value for methanol is less than half of that for the hydrocarbon fuels. 

All of the fuels considered here are gaseous fuels. In the case of a liquid fuel, the heating values would be lower than those for the corresponding vapor fuel by the amount of energy needed to vaporize the fuel at the reference temperature and pressure: i.e., by the enthalpy of vaporization of the fuel. An example is provided in Example 2.4 of Chapter 2 of Turns 4th ed.